In [1]:
# Loading Data

%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
import random


# using the SQLite Table to read data.
con = sqlite3.connect('final.sqlite') 



#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
cleaned_data = pd.read_sql_query("""
SELECT *
FROM Reviews
""", con) 
#select a random 150 rows 

data_pos = cleaned_data[cleaned_data["Score"]=='positive'].sample(n = 5000, replace = True)
data_neg = cleaned_data[cleaned_data["Score"]=='negative'].sample(n = 5000,replace = True)
final_data = pd.concat([data_pos, data_neg])
cleaned_data['Score'].value_counts()

positive    38479
negative     7592
Name: Score, dtype: int64

In [2]:
# Sorting data based on time
# convert time into understnad time
final_data['Time'] = pd.to_datetime(final_data['Time'], unit = 's')
final_data

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
24718,29072,31701,B001MURXMK,AH15LB0KISOK2,Sarahnarizona,0,0,positive,2012-02-04,Wow this is a ton of yummy cookies....,First and for most 4.6 lbs of cookies is A LOT...,b'first lbs cooki lot cooki individu packag co...
15836,21724,23782,B0013NUGDE,A297EGCHDYRAV7,"Lois I. Bravo ""lbravo""",0,0,positive,2012-05-24,Popchips Sweet Potatoe,Love the taste. They are low in calories and ...,b'love tast low calori better chip satisfi swe...
9094,37847,41104,B000I05WLY,A383XURHVF8ON6,Mrs. Kilo Delta,6,6,positive,2007-09-24,Oh happy day! amazon carries this!,"This stuff is great, adds a special touch to m...",b'stuff great add special touch chocol cake su...
9836,9010,9866,B000KJRF40,A3C6BIUJS018BC,"Carolyn G. Gustavich ""Noelkimberly""",1,1,positive,2009-10-11,liquor filled chocolates,I bought this as a gift. He liked it very muc...,b'bought gift like much came good condit good ...
32374,18216,19854,B00390T5JA,A3VU2J7G0U35AU,Jaybird,3,3,positive,2011-12-30,"Hot, Perfect Tea, Anyone?",I ordered this lovely machine for myself for C...,b'order love machin christma get tire endless ...
32134,10620,11595,B0037B0A7M,A2ZOPSZMJO8WH8,L. Arango,2,2,positive,2011-07-23,Outstanding Service and Quality!,I live overseas and bought this for a friend l...,b'live oversea bought friend live usa babi rec...
3791,30945,33712,B000CONMBS,A2TBJE5RIDY6H0,Birdie,0,0,positive,2011-11-07,Love it!,This stuff is great. Healthy for children. G...,b'stuff great healthi children great tast buy ...
725,10118,11051,B0000VYKXC,A3ON2GDK2WEM7Z,"Mary L. Zimmer ""kaseysmom45""",0,2,positive,2007-01-11,Gifts of Coffee,I sent this coffee to two friends back east. ...,b'sent coffe two friend back east thorough enj...
4536,21581,23627,B000E1FXLG,A2TQYRDGMDAK18,Deanna K. Kovach,3,3,positive,2008-02-08,SF Pistachio Pudding Mix,I make a great Watergate Salad (from the Jell-...,b'make great waterg salad websit product leav ...
25396,43579,47421,B001QE7S4K,A35R32TA60XD57,M. Torma,0,0,positive,2011-04-08,Exceptional dog food!,I just started feeding my cats this brand of f...,b'start feed cat brand food impress compani th...


In [3]:
final_data = final_data.sort_values(by='Time')

In [4]:
X = final_data['CleanedText']
print('shape of X:', X.shape)

shape of X: (10000,)


In [5]:
# class label
y = final_data["Score"]
print('shape of y:', y.shape)

shape of y: (10000,)


In [6]:
# split data into train and test where 70% data used to train model and 30% for test
# final_data[:int(len(final_data) * 0.75)], final_data[int(len(final_data) * 0.75):]
from sklearn.model_selection import train_test_split
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
print(X_train.shape, y_train.shape, x_test.shape)

(7000,) (7000,) (3000,)


In [7]:
X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)
x_test = np.nan_to_num(x_test)
y_test = np.nan_to_num(y_test)

In [8]:
# Train Vectorizor
from sklearn.feature_extraction.text import CountVectorizer 

bow = CountVectorizer()
X_train = bow.fit_transform(X_train)
X_train

<7000x10442 sparse matrix of type '<class 'numpy.int64'>'
	with 228109 stored elements in Compressed Sparse Row format>

In [9]:
x_test = bow.transform(x_test)
x_test

<3000x10442 sparse matrix of type '<class 'numpy.int64'>'
	with 94801 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [11]:
pred = clf.predict(x_test)

In [12]:
from sklearn import metrics
metrics.accuracy_score(y_test, pred)

0.845

In [14]:
from sklearn.model_selection import GridSearchCV
def SVM(X_train, y_train, x_test, y_test):
    parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
    svc = svm.SVC(gamma='scale')
    clf = GridSearchCV(svc, parameters, cv=10)
    clf.fit(X_train, y_train)
    print(clf.best_estimator_)
    print('The score with gridSearchCV is:'+ str(clf.score(x_test, y_test)))


In [16]:
SVM(X_train, y_train, x_test, y_test)


SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
The score with gridSearchCV is:0.882


In [28]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
def SVMR(X_train, y_train, x_test, y_test):
    random_parameters = dict(C=[1,10, 100, 1000], gamma=[1e-3, 1e-4])
    clf = RandomizedSearchCV(SVC(),random_parameters, cv=10)
    clf.fit(X_train, y_train)
    print(clf.best_estimator_)
    print('The score with randomizedserach is: '+ str(clf.score(x_test, y_test)))
    
    

In [29]:
SVMR(X_train, y_train, x_test, y_test)

/usr/local/anaconda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
The score with randomizedserach is: 0.8906666666666667
